In [1]:
import re
import json
import requests
from selenium import webdriver
import geckodriver_autoinstaller
import pandas as pd
import time

In [2]:
#Hemos generado un archivo para poder guardar aquí la contraseña e usuario
with open('credentials.json') as f:
    credentials = json.load(f)
    
usuario = credentials['usuario']
password=credentials['password']

In [3]:
from linkedin_api import Linkedin
# Nos autenticamos en Linkedin
api = Linkedin(usuario, password)

In [4]:
jobs = api.search_jobs(keywords='Intern', companies=None, experience=None, job_type=None, job_title=None, industries=None, location_name='Barcelona', remote=True, listed_at=86400, limit=10, offset=0)

In [5]:
dir(api)

['_MAX_REPEATED_REQUESTS',
 '_MAX_SEARCH_COUNT',
 '_MAX_UPDATE_COUNT',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_fetch',
 '_post',
 'client',
 'get_company',
 'get_company_updates',
 'get_conversation',
 'get_conversation_details',
 'get_conversations',
 'get_current_profile_views',
 'get_invitations',
 'get_profile',
 'get_profile_connections',
 'get_profile_contact_info',
 'get_profile_member_badges',
 'get_profile_network_info',
 'get_profile_privacy_settings',
 'get_profile_skills',
 'get_profile_updates',
 'get_school',
 'get_user_profile',
 'logger',
 'mark_conversation_as_seen',
 'remove_connection',
 'reply_invitation',
 'search',
 'search_companies',
 

In [6]:
help(api.get_company)

Help on method get_company in module linkedin_api.linkedin:

get_company(public_id) method of linkedin_api.linkedin.Linkedin instance
    Fetch data about a given LinkedIn company.
    
    :param public_id: LinkedIn public ID for a company
    :type public_id: str
    
    :return: Company data
    :rtype: dict



In [7]:
jobs_df = pd.DataFrame(jobs)
jobs_df.to_csv('Jobs.csv')

In [8]:
#Con Regex voy a extraer los ids de las empresas
company_reference = r'([0-9]+)'

In [9]:
#Los almacenaré en CompanyId
jobs_df['CompanyId']=None

In [10]:
index_description= jobs_df.columns.get_loc('companyDetails')
index_date = jobs_df.columns.get_loc('CompanyId')
print(index_description, index_date)

2 18


In [11]:
jobs_df['JobId']=None
jobs_df['companyDetails']=jobs_df['companyDetails'].astype(str)

In [12]:
#Con un for hago el regex en toda la columna y genero la nueva
for row in range(0,len(jobs_df)):
        if re.search(company_reference, jobs_df.iat[row,index_description]) != None:
            reference = re.search(company_reference, jobs_df.iat[row,index_description]).group()
            jobs_df.iat[row, index_date] = reference
        else:
            reference = None

In [13]:
#La almaceno en una lista visto que no la puedo trabajar desde el df
companies = list(jobs_df['CompanyId'])

In [14]:
#Hago lo mismo con el job reference
job_reference = r'([0-9]+)'

In [15]:
#Definiremos los indices de estas columnas
index_description= jobs_df.columns.get_loc('*savingInfo')
index_date = jobs_df.columns.get_loc('JobId')
print(index_description, index_date)

9 19


In [16]:
for row in range(0,len(jobs_df)):
        if re.search(job_reference, jobs_df.iat[row,index_description]) != None:
            reference = re.search(job_reference, jobs_df.iat[row,index_description]).group()
            jobs_df.iat[row, index_date] = reference
        else:
            reference = None

In [17]:
jobs_df

,salaryInsights,dashEntityUrn,companyDetails,jobState,title,duration,entityUrn,briefBenefitsDescription,applyMethod,*savingInfo,new,formattedLocation,expireAt,listedAt,$recipeTypes,$type,standardizedAddresses,sourceDomain,CompanyId,JobId
0,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2789801957,{'company': 'urn:li:fs_normalized_company:1619...,LISTED,Intern Masters Business (51006201),None,urn:li:fs_normalized_jobPosting:2789801957,,"{'applyStartersPreferenceVoid': True, 'company...",urn:li:fs_jobSavingInfo:2789801957,False,"Barcelona, Catalonia, Spain",1639146072000,1636554073000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,dxctechnology.wd1.myworkdayjobs.com,16198010,2789801957
1,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2768736549,{'company': 'urn:li:fs_normalized_company:9262...,LISTED,People Controlling Intern (They/She/He),None,urn:li:fs_normalized_jobPosting:2768736549,,"{'applyStartersPreferenceVoid': True, 'company...",urn:li:fs_jobSavingInfo:2768736549,False,"Barcelona, Catalonia, Spain",1640034235000,1636717243000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,app.greenhouse.io,9262901,2768736549
2,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2801338737,{'company': 'urn:li:fs_normalized_company:2901...,LISTED,Business Operations Intern,None,urn:li:fs_normalized_jobPosting:2801338737,,{'easyApplyUrl': 'https://www.linkedin.com/job...,urn:li:fs_jobSavingInfo:2801338737,True,"Barcelona, Catalonia, Spain",1652883049000,1637331077000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,None,29016729,2801338737
3,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2759299909,{'company': 'urn:li:fs_normalized_company:3226...,LISTED,Strategy Business Analytics Intern (Spain based),None,urn:li:fs_normalized_jobPosting:2759299909,,"{'applyStartersPreferenceVoid': True, 'company...",urn:li:fs_jobSavingInfo:2759299909,False,"Barcelona, Catalonia, Spain",1639843935000,1636986295000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,None,3226972,2759299909
4,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2659225588,{'company': 'urn:li:fs_normalized_company:1115...,LISTED,SAP iXp Intern - Digital Demand Strategist,None,urn:li:fs_normalized_jobPosting:2659225588,,"{'applyStartersPreferenceVoid': True, 'company...",urn:li:fs_jobSavingInfo:2659225588,False,"Barcelona, Catalonia, Spain",1642549517000,1636198129000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,jobs.sap.com,1115,2659225588
5,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2801345488,{'company': 'urn:li:fs_normalized_company:1149...,LISTED,Full time internship - Consulting/Business Ana...,None,urn:li:fs_normalized_jobPosting:2801345488,,"{'unifyApplyEnabled': True, 'type': 'CONTACT_I...",urn:li:fs_jobSavingInfo:2801345488,True,Greater Madrid Metropolitan Area,1639923934000,1637331935000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,None,11491309,2801345488
6,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2751447440,{'company': 'urn:li:fs_normalized_company:2458...,LISTED,Performance Management / MI Intern,None,urn:li:fs_normalized_jobPosting:2751447440,,"{'applyStartersPreferenceVoid': True, 'company...",urn:li:fs_jobSavingInfo:2751447440,False,"Barcelona, Catalonia, Spain",1639302850000,1635933617000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,www.careers.zurich.com,2458,2751447440
7,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2784483258,{'company': 'urn:li:fs_normalized_company:3720...,LISTED,FP&A AX Intern,None,urn:li:fs_normalized_jobPosting:2

In [18]:
#Miro que contiene esta funcionalidad de la api y veo que lo que me interesa es el name
api.get_company(2289716)

{'staffingCompany': False,
 'companyIndustries': [{'localizedName': 'Leisure, Travel & Tourism',
   'entityUrn': 'urn:li:fs_industry:30'}],
 'callToAction': {'callToActionType': 'VIEW_WEBSITE',
  'visible': True,
  'callToActionMessage': {'textDirection': 'USER_LOCALE',
   'text': 'Visit website'},
  'url': 'http://www.edreamsodigeo.com'},
 'staffCount': 1428,
 'adsRule': 'STANDARD',
 'companyEmployeesSearchPageUrl': 'https://www.linkedin.com/vsearch/p?f_CC=2289716',
 'viewerFollowingJobsUpdates': False,
 'staffCountRange': {'start': 1001, 'end': 5000},
 'permissions': {'landingPageAdmin': False,
  'admin': False,
  'adAccountHolder': False},
 'logo': {'image': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 200,
      'fileIdentifyingUrlPathSegment': '200_200/0/1636636337161?e=1645660800&v=beta&t=v252swbk7Y-EyML6C4H_84uW6-eLO-JpKKV3hv26E2c',
      'expiresAt': 1645660800000,
      'height': 200},
     {'width': 100,
      'fileIdentifyingUrlPathSegment': '100_100/0/163663

In [19]:
#Miro como extraer el nombre
company_n = api.get_company(2289716)
pd_company= pd.json_normalize(company_n)
pd_company['name'][0]

'eDreams ODIGEO'

In [20]:
# Añado los nombres de las empresas a un dataframe
companies_df=[]
for id in companies:
    company = api.get_company(id)
    pd_company= pd.json_normalize(company)
    company_name = pd_company['name'][0]
    reference = id
    companies_df.append(
    {'Company':company_name,
     'CompanyId': reference
    })

In [21]:
companies=pd.DataFrame(companies_df)

In [22]:
#Uno mi dataframe con el que ya tenía anteriormente
jobs_final = pd.merge(companies, jobs_df, on='CompanyId',how='outer')

In [23]:
jobs_final

,Company,CompanyId,salaryInsights,dashEntityUrn,companyDetails,jobState,title,duration,entityUrn,briefBenefitsDescription,...,*savingInfo,new,formattedLocation,expireAt,listedAt,$recipeTypes,$type,standardizedAddresses,sourceDomain,JobId
0,DXC Technology,16198010,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2789801957,{'company': 'urn:li:fs_normalized_company:1619...,LISTED,Intern Masters Business (51006201),None,urn:li:fs_normalized_jobPosting:2789801957,,...,urn:li:fs_jobSavingInfo:2789801957,False,"Barcelona, Catalonia, Spain",1639146072000,1636554073000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,dxctechnology.wd1.myworkdayjobs.com,2789801957
1,Glovo,9262901,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2768736549,{'company': 'urn:li:fs_normalized_company:9262...,LISTED,People Controlling Intern (They/She/He),None,urn:li:fs_normalized_jobPosting:2768736549,,...,urn:li:fs_jobSavingInfo:2768736549,False,"Barcelona, Catalonia, Spain",1640034235000,1636717243000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,app.greenhouse.io,2768736549
2,BRICKBRO,29016729,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2801338737,{'company': 'urn:li:fs_normalized_company:2901...,LISTED,Business Operations Intern,None,urn:li:fs_normalized_jobPosting:2801338737,,...,urn:li:fs_jobSavingInfo:2801338737,True,"Barcelona, Catalonia, Spain",1652883049000,1637331077000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,None,2801338737
3,Typeform,3226972,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2759299909,{'company': 'urn:li:fs_normalized_company:3226...,LISTED,Strategy Business Analytics Intern (Spain based),None,urn:li:fs_normalized_jobPosting:2759299909,,...,urn:li:fs_jobSavingInfo:2759299909,False,"Barcelona, Catalonia, Spain",1639843935000,1636986295000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,None,2759299909
4,SAP,1115,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2659225588,{'company': 'urn:li:fs_normalized_company:1115...,LISTED,SAP iXp Intern - Digital Demand Strategist,None,urn:li:fs_normalized_jobPosting:2659225588,,...,urn:li:fs_jobSavingInfo:2659225588,False,"Barcelona, Catalonia, Spain",1642549517000,1636198129000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,jobs.sap.com,2659225588
5,IQVIA Spain,11491309,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2801345488,{'company': 'urn:li:fs_normalized_company:1149...,LISTED,Full time internship - Consulting/Business Ana...,None,urn:li:fs_normalized_jobPosting:2801345488,,...,urn:li:fs_jobSavingInfo:2801345488,True,Greater Madrid Metropolitan Area,1639923934000,1637331935000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,None,2801345488
6,Zurich Insurance Company Ltd,2458,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2751447440,{'company': 'urn:li:fs_normalized_company:2458...,LISTED,Performance Management / MI Intern,None,urn:li:fs_normalized_jobPosting:2751447440,,...,urn:li:fs_jobSavingInfo:2751447440,False,"Barcelona, Catalonia, Spain",1639302850000,1635933617000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,www.careers.zurich.com,2751447440
7,Criteo,37209,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2784483258,{'company': 'urn:li:fs_normalized_company:3720...,LISTED,FP&A AX Intern,None,urn:li:fs_normalized_jobPosting:2784483258,,...,urn:li:fs_jobSavingInfo:2784483258,False,"Barcelona, Catalonia, Spain",1638991434000,1636399435000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,careers.criteo.com,2784483258
8

# Scrapping:

La idea es scrapear las ofertas una por una para extraer la descripción y añadirla al dataframe. 

In [24]:
geckodriver_autoinstaller.install()

'/usr/local/lib/python3.9/site-packages/geckodriver_autoinstaller/v0.30.0/geckodriver'

In [25]:
driver = webdriver.Firefox()

In [26]:
jobs_df.astype({'JobId': 'int32'}).dtypes
jobs = list(jobs_df['JobId'])

In [27]:
#Consigo la lista de JobIds
jobs

['2789801957',
 '2768736549',
 '2801338737',
 '2759299909',
 '2659225588',
 '2801345488',
 '2751447440',
 '2784483258',
 '2755937274',
 '2796330822']

In [28]:
driver.get('http://linkedin.com/')

In [29]:
usuario_ref = driver.find_element(by='id',value='session_key')
password_ref = driver.find_element(by='id',value='session_password')

login = driver.find_element(by = 'css selector',value = '.sign-in-form__submit-button')

In [30]:
usuario_ref.send_keys(usuario)

In [31]:
password_ref.send_keys(password)

In [32]:
login.click()

In [34]:
#Hago un scrapping de todos los links generados de la jobID

jobs_complete=[]

for reference in jobs:
    job_url=f'https://www.linkedin.com/jobs/search/?currentJobId={reference}'
    print(job_url)
    try:
        driver.get(job_url)
        ele = driver.find_element(by='id', value = 'job-details')
        JobId = reference
        jobs_complete.append(
        {'job_description': ele.text,
         'JobId': JobId
        })
        time.sleep(20)
    except:
        print("Not possible")

https://www.linkedin.com/jobs/search/?currentJobId=2789801957
https://www.linkedin.com/jobs/search/?currentJobId=2768736549
https://www.linkedin.com/jobs/search/?currentJobId=2801338737
Not possible
https://www.linkedin.com/jobs/search/?currentJobId=2759299909
https://www.linkedin.com/jobs/search/?currentJobId=2659225588
https://www.linkedin.com/jobs/search/?currentJobId=2801345488
Not possible
https://www.linkedin.com/jobs/search/?currentJobId=2751447440
Not possible
https://www.linkedin.com/jobs/search/?currentJobId=2784483258
https://www.linkedin.com/jobs/search/?currentJobId=2755937274
https://www.linkedin.com/jobs/search/?currentJobId=2796330822


In [35]:
#Genero un dataframe con esta información
jobs_description = pd.DataFrame(jobs_complete)
jobs_description

,job_description,JobId
0,Essential Job Functions\nAssists the project t...,2789801957
1,About Glovo\n\nWe’re a Barcelona-based startup...,2768736549
2,"About Typeform\n\nOne sunny Barcelona morning,...",2759299909
3,What We Offer\n\nOur company culture is focuse...,2659225588
4,,2784483258
5,"Description\n\nLocation: EMEA (LU, UK, DE, FR,...",2755937274
6,Buscamos un becario para reforzar y ampliar nu...,2796330822


In [36]:
#Lo uno con mi anterior dataframe
jobs_final= jobs_description.merge(jobs_final,on = 'JobId' , how='outer')

jobs_final

,job_description,JobId,Company,CompanyId,salaryInsights,dashEntityUrn,companyDetails,jobState,title,duration,...,applyMethod,*savingInfo,new,formattedLocation,expireAt,listedAt,$recipeTypes,$type,standardizedAddresses,sourceDomain
0,Essential Job Functions\nAssists the project t...,2789801957,DXC Technology,16198010,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2789801957,{'company': 'urn:li:fs_normalized_company:1619...,LISTED,Intern Masters Business (51006201),None,...,"{'applyStartersPreferenceVoid': True, 'company...",urn:li:fs_jobSavingInfo:2789801957,False,"Barcelona, Catalonia, Spain",1639146072000,1636554073000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,dxctechnology.wd1.myworkdayjobs.com
1,About Glovo\n\nWe’re a Barcelona-based startup...,2768736549,Glovo,9262901,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2768736549,{'company': 'urn:li:fs_normalized_company:9262...,LISTED,People Controlling Intern (They/She/He),None,...,"{'applyStartersPreferenceVoid': True, 'company...",urn:li:fs_jobSavingInfo:2768736549,False,"Barcelona, Catalonia, Spain",1640034235000,1636717243000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,app.greenhouse.io
2,"About Typeform\n\nOne sunny Barcelona morning,...",2759299909,Typeform,3226972,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2759299909,{'company': 'urn:li:fs_normalized_company:3226...,LISTED,Strategy Business Analytics Intern (Spain based),None,...,"{'applyStartersPreferenceVoid': True, 'company...",urn:li:fs_jobSavingInfo:2759299909,False,"Barcelona, Catalonia, Spain",1639843935000,1636986295000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,None
3,What We Offer\n\nOur company culture is focuse...,2659225588,SAP,1115,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2659225588,{'company': 'urn:li:fs_normalized_company:1115...,LISTED,SAP iXp Intern - Digital Demand Strategist,None,...,"{'applyStartersPreferenceVoid': True, 'company...",urn:li:fs_jobSavingInfo:2659225588,False,"Barcelona, Catalonia, Spain",1642549517000,1636198129000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,jobs.sap.com
4,,2784483258,Criteo,37209,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2784483258,{'company': 'urn:li:fs_normalized_company:3720...,LISTED,FP&A AX Intern,None,...,"{'applyStartersPreferenceVoid': True, 'company...",urn:li:fs_jobSavingInfo:2784483258,False,"Barcelona, Catalonia, Spain",1638991434000,1636399435000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,careers.criteo.com
5,"Description\n\nLocation: EMEA (LU, UK, DE, FR,...",2755937274,Amazon,1586,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2755937274,{'company': 'urn:li:fs_normalized_company:1586...,LISTED,Business Intelligence Intern - Spain,None,...,"{'applyStartersPreferenceVoid': True, 'company...",urn:li:fs_jobSavingInfo:2755937274,False,"Barcelona, Catalonia, Spain",1639826367000,1636457147000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,www.amazon.jobs
6,Buscamos un becario para reforzar y ampliar nu...,2796330822,"Linde Material Handling Iberica, SA",1082030,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2796330822,{'company': 'urn:li:fs_normalized_company:1082...,LISTED,Used Truck Intern,None,...,{'easyApplyUrl': 'https://www.linkedin.com/job...,urn:li:fs_jobSavingInfo:2796330822,False,"Pallejà, Catalonia, Spain",1639320086000,1636728086000,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,None,None
7,NaN,2801338737,BRICKBRO,29016729,"{'lockModuleShown': True, 'salaryExplorerUrl':...",urn:li:fsd_jobPosting:2801338737,{'company': 'ur

In [37]:
#Añado la parte que falta para conseguir la url de acceso directo a la oferta
jobs_final['Job_link']='https://www.linkedin.com/jobs/search/?currentJobId='+jobs_final['JobId']

In [38]:
#Escojo las columnas que me interesan
jobs_fin = jobs_final[['title','formattedLocation','Company','job_description','Job_link']]

In [39]:
jobs_fin

,title,formattedLocation,Company,job_description,Job_link
0,Intern Masters Business (51006201),"Barcelona, Catalonia, Spain",DXC Technology,Essential Job Functions\nAssists the project t...,https://www.linkedin.com/jobs/search/?currentJ...
1,People Controlling Intern (They/She/He),"Barcelona, Catalonia, Spain",Glovo,About Glovo\n\nWe’re a Barcelona-based startup...,https://www.linkedin.com/jobs/search/?currentJ...
2,Strategy Business Analytics Intern (Spain based),"Barcelona, Catalonia, Spain",Typeform,"About Typeform\n\nOne sunny Barcelona morning,...",https://www.linkedin.com/jobs/search/?currentJ...
3,SAP iXp Intern - Digital Demand Strategist,"Barcelona, Catalonia, Spain",SAP,What We Offer\n\nOur company culture is focuse...,https://www.linkedin.com/jobs/search/?currentJ...
4,FP&A AX Intern,"Barcelona, Catalonia, Spain",Criteo,,https://www.linkedin.com/jobs/search/?currentJ...
5,Business Intelligence Intern - Spain,"Barcelona, Catalonia, Spain",Amazon,"Description\n\nLocation: EMEA (LU, UK, DE, FR,...",https://www.linkedin.com/jobs/search/?currentJ...
6,Used Truck Intern,"Pallejà, Catalonia, Spain","Linde Material Handling Iberica, SA",Buscamos un becario para reforzar y ampliar nu...,https://www.linkedin.com/jobs/search/?currentJ...
7,Business Operations Intern,"Barcelona, Catalonia, Spain",BRICKBRO,NaN,https://www.linkedin.com/jobs/search/?currentJ...
8,Full time internship - Consulting/Business Ana...,Greater Madrid Metropolitan Area,IQVIA Spain,NaN,https://www.linkedin.com/jobs/search/?currentJ...
9,Performance Management / MI Intern,"Barcelona, Catalonia, Spain",Zurich Insurance Company Ltd,NaN,https://www.linkedin.com/jobs/search/?currentJ...
